In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from openai import OpenAI
import os
# import openpyxl
import base64
import json
from PIL import Image
from io import BytesIO
import pandas as pd
from configparser import ConfigParser
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import random

In [ ]:
sheet_name = 'lesion' # /'normal'/'var_normal'
file_path = 'clinical_data.xlsx'
data_df = pd.read_excel(file_path,sheet_name=sheet_name)

In [ ]:
def convert_to_dict(df, key_column):
    """
    Converts a DataFrame into a dictionary with a specified column as the master key.

    Parameters:
        df (pd.DataFrame): The DataFrame to convert.
        key_column (str): The column to use as the master key.

    Returns:
        dict: The transformed dictionary.
    """
    # Convert DataFrame to dictionary
    data_dict = df.set_index(key_column).to_dict(orient='index')
    return data_dict

# Convert to dictionary using 'SMITA_ID' as the key
data_dict = convert_to_dict(train_df, 'SMITA_ID')

In [ ]:
cot_dict = {}
for smita_id in tqdm(train_dict.keys()):
  file_path = data_dict[smita_id]['file_path']
  if sheet_name != 'normal':
    location = data_dict[smita_id]['Location']
    colour = data_dict[smita_id]['Color']
    margin = data_dict[smita_id]['Margin']
    texture = data_dict[smita_id]['Surface feature']
    scrappability = data_dict[smita_id]['Description of the Lesion']
    cot_updated_template = f"Upon inspecting the {location}, we observe a {colour} appearance contrast with adjacent regions and {margin} edges and a {texture} surface. Based on all the above observations we can conclude that this is abnormal."
    temp_cot = cot_updated_template.format(location=location, colour=colour, margin=margin, texture=texture)
  else:
    temp_cot = "Upon inspecting the right buccal mucosa, we observe a pale pink and well-defined  edges and a smooth and soft surface. Based on all the above observations we can conclude that this is normal."
  cot_dict[file_path] = temp_cot

In [ ]:
file_name = f"{sheet_name}_cot_dict.json"
with open(file_name, "w", encoding="utf-8") as json_file:
    json.dump(cot_dict, json_file, indent=4, ensure_ascii=False)